In [1]:
### Import libraries

import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

# For recommendation systems specifically
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD  # for matrix factorization

# If using collaborative filtering
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# Warnings control
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%matplotlib inline


In [2]:
### load data

df = pd.read_csv("train.csv")

In [6]:
### EDA

df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.shape

(891, 12)

In [8]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [11]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
# 1. Handle missing
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df.drop('Cabin', axis=1, inplace=True)

# 2. Feature engineering
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.')
df['Family_Size'] = df['SibSp'] + df['Parch'] + 1

# 3. Encode
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 4. Drop
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [13]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size
0,0,3,0,22.0,1,0,7.2500,0,Mr,2
1,1,1,1,38.0,1,0,71.2833,1,Mrs,2
2,1,3,1,26.0,0,0,7.9250,0,Miss,1
3,1,1,1,35.0,1,0,53.1000,0,Mrs,2
4,0,3,0,35.0,0,0,8.0500,0,Mr,1


In [15]:
df.shape

(891, 10)

In [16]:
print(df['Title'].value_counts())

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Ms            1
Mme           1
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64


In [17]:
# Group rare titles (< 10 occurrences)
df['Title'] = df['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Mlle', 
                                     'Ms', 'Mme', 'Don', 'Lady', 'Sir', 
                                     'Capt', 'Countess', 'Jonkheer'], 'Rare')

# Encode
title_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4}
df['Title'] = df['Title'].map(title_mapping)

# Check
print(df['Title'].value_counts())
print(df.dtypes)  # All numeric now?

Title
0    517
1    182
2    125
3     40
4     27
Name: count, dtype: int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked         int64
Title            int64
Family_Size      int64
dtype: object


In [18]:
### Model creation - Standard Regression

# Separate features and target
X = df.drop('Survived', axis=1)
y = df['Survived']

# Train/dev split
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression (baseline)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_dev)
print(f"Accuracy: {accuracy_score(y_dev, y_pred):.3f}")

Accuracy: 0.782


In [19]:
### Model creation - Random Forest

from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred_rf = rf_model.predict(X_dev)
print(f"Random Forest Accuracy: {accuracy_score(y_dev, y_pred_rf):.3f}")

Random Forest Accuracy: 0.838


In [20]:
### Kaggle submission

In [21]:
# 1. Load test data
test = pd.read_csv('test.csv')

# 2. Apply SAME preprocessing
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)  # Test might have missing Fare
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)
test.drop('Cabin', axis=1, inplace=True)

test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.')
test['Family_Size'] = test['SibSp'] + test['Parch'] + 1

test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})
test['Embarked'] = test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Encode Title (same mapping!)
test['Title'] = test['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Mlle', 
                                        'Ms', 'Mme', 'Don', 'Lady', 'Sir', 
                                        'Capt', 'Countess', 'Jonkheer'], 'Rare')
test['Title'] = test['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4})

# Save PassengerId before dropping
passenger_ids = test['PassengerId']

# Drop columns
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

# 3. Predict
predictions = rf_model.predict(test)

# 4. Create submission file
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})

submission.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!
